In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from scipy import sparse 
from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
from py_pcha import PCHA

In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  


In [ ]:
outdirfig = "/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/figures/250407_zscore"
!mkdir -p $outdirfig



# load gene annotation

In [ ]:
gene_modules = GeneModules()
anno, color, gene_styled = gene_modules.check_genes('Cdh13')
print("\t".join(anno))
print("\t".join(color))
print("\t".join(gene_styled))

In [ ]:
# AC genes
f = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/Saumya_P6-21_AC_genes.csv'
df = pd.read_csv(f)

df_a = df.iloc[:25]
df_c = df.iloc[25:]

alltime_a = np.unique(df_a.values)
alltime_c = np.unique(df_c.values)
alltime_ac = np.hstack([alltime_a, alltime_c])

ac_overlap = np.intersect1d(alltime_a, alltime_c)

print(df_a.shape, df_c.shape, alltime_a.shape, alltime_c.shape, alltime_ac.shape, ac_overlap.shape)
df.head()

In [ ]:
# use those 286 genes
# df = pd.read_csv("../../data/cheng21_cell_scrna/res/candidate_genes_vincent_0503_v2.csv")
df = pd.read_csv("../../data/cheng21_cell_scrna/res/L23-ABC-genes-n288-n286unq-annot.csv")
genes_l23 = df['gene'].astype(str).values
genes_l23a = df[df['P17on']=='A']['gene'].astype(str).values
genes_l23b = df[df['P17on']=='B']['gene'].astype(str).values
genes_l23c = df[df['P17on']=='C']['gene'].astype(str).values

print(genes_l23a.shape, genes_l23b.shape, genes_l23c.shape)
genes_grp = df['P17on'].astype(str).values
assert len(genes_l23) == len(np.unique(genes_l23))

genes_l23.shape

# load data 

In [ ]:
# f_anndata_in  = "../../data/v1_multiome/superdupermegaRNA_hasraw_multiome_l23.h5ad"# _multiome_l23.h5ad"#  L23_allmultiome_raw.h5ad"
f_anndata_in  = "../../data/v1_multiome/L23_allmultiome_raw.h5ad"
adata = anndata.read(f_anndata_in)
adata

In [ ]:
sample_labels = ["-".join(cell.split(' ')[0].split('-')[2:]).replace('-2023', '') for cell in adata.obs.index]
time_labels = [s[:-1].replace('DR', '') for s in sample_labels]

adata.obs['n_counts'] = adata.obs['nCount_RNA']
adata.obs['sample'] = sample_labels
adata.obs['time']   = time_labels

uniq_samples = natsorted(np.unique(sample_labels))
uniq_times = natsorted(np.unique(time_labels))

nr_samples = [s for s in uniq_samples if "DR" not in s]
dr_samples = [s for s in uniq_samples if "DR" in s]
print(uniq_times)
print(nr_samples)
print(dr_samples)

# adata.obs['sample'] = sample_labels

In [ ]:
# select samples
# adata = adata[adata.obs['sample'].isin(nr_samples)]
# adata = adata[adata.obs['time'].isin(['P6', 'P8', 'P10', 'P12', 'P14', 'P17', 'P21'])]
adata.obs['cond'] = adata.obs['sample'].apply(lambda x: x[:-1]) # .unique()

# remove mitocondria genes
adata = adata[:,~adata.var['features'].str.contains(r'^mt-')]

# select
# adata = adata[adata.obs['cond'].str.contains(r"NR$")]
adata.obs['sample'].unique(), adata.obs['cond'].unique()

In [ ]:
# define
# genes = adata.var.index.values
conds = adata.obs['cond'].values
types = adata.obs['Type'].values
samps = adata.obs['sample'].values

# organize
rename = {
    "L2/3_A": "L2/3_A",
    "L2/3_B": "L2/3_B",
    "L2/3_C": "L2/3_C",
    
    "L2/3_1": "L2/3_A",
    "L2/3_2": "L2/3_B",
    "L2/3_3": "L2/3_C",
    
    "L2/3_AB": "L2/3_A",
    "L2/3_BC": "L2/3_C",
}
adata.obs['easitype'] = adata.obs['Type'] #.apply(lambda x: rename[x])
adata

In [ ]:
adata.obs['Type'].unique()

In [ ]:
# filter genes
cond = np.ravel((adata.X>0).sum(axis=0)) > 10 # expressed in more than 10 cells
adata = adata[:,cond]
genes = adata.var.index.values

# counts
x = adata.X
cov = adata.obs['n_counts'].values
genes = adata.var.index.values

# CP10k
# xn = x/cov.reshape(x.shape[0], -1)*1e4
xn = (sparse.diags(1/cov).dot(x))*1e4

# log10(CP10k+1)
xln = xn.copy()
xln.data = np.log10(xln.data+1)

adata.layers[    'norm'] = np.array(xn.todense())
adata.layers[ 'lognorm'] = np.array(xln.todense())
adata.layers['zlognorm'] = zscore(np.array(xln.todense()), axis=0)

In [ ]:
# select HVGs with mean and var
nbin = 20
qth = 0.3

# min
gm = np.ravel(xn.mean(axis=0))

# var
tmp = xn.copy()
tmp.data = np.power(tmp.data, 2)
gv = np.ravel(tmp.mean(axis=0))-gm**2

# cut 
lbl = pd.qcut(gm, nbin, labels=np.arange(nbin))
gres = pd.DataFrame()
gres['name'] = genes
gres['lbl'] = lbl
gres['mean'] = gm
gres['var'] = gv
gres['ratio']= gv/gm

# select
gres_sel = gres.groupby('lbl')['ratio'].nlargest(int(qth*(len(gm)/nbin))) #.reset_index()
gsel_idx = np.sort(gres_sel.index.get_level_values(1).values)
assert np.all(gsel_idx != -1)


In [ ]:
fig, axs = plt.subplots(1,2, figsize=(6*2,4), sharex=True, sharey=True)
ax = axs[0]
ax.scatter(gm, gv/gm, s=5, edgecolor='none', color='gray')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('mean expr (CP10k)')
ax.set_ylabel('var/mean')
ax.set_title(f'{len(gm):,} genes')

ax = axs[1]
ax.scatter(gm, gv/gm, s=5, edgecolor='none', color='gray')
ax.scatter(gm[gsel_idx], (gv/gm)[gsel_idx], c=lbl[gsel_idx], s=5, edgecolor='none', cmap='viridis_r', label=f'{len(gsel_idx):,} HVGs')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('mean expr (CP10k)')
ax.set_ylabel('var/mean')
ax.legend(bbox_to_anchor=(1,1))

plt.show()

In [ ]:
adata_hvg = adata[:,gsel_idx]
print(adata_hvg.shape)
genes_sel = adata_hvg.var.index.values

adata_hvg_early = adata_hvg[adata_hvg.obs['cond'].isin(['P6'])]
adata_hvg_later = adata_hvg[adata_hvg.obs['cond'].isin(['P21'])]


pca_early = PCA(n_components=5)
pca_early.fit(adata_hvg_early.layers['zlognorm'][...]) #
pcs_early = pca_early.transform(adata_hvg.layers['zlognorm'][...])

# pcs_early[:,0] = -pcs_earl[:,0] # flip PC1
# pcs_early[:,2] = -pcs_early[:,2] # flip PC3

# learn on parts, project for everything
pca_later = PCA(n_components=5)
pca_later.fit(adata_hvg_later.layers['zlognorm'][...]) #
pcs_later = pca_later.transform(adata_hvg.layers['zlognorm'][...])

pcs_later[:,0] = -pcs_later[:,0] # flip PC1
pcs_later[:,1] = -pcs_later[:,1] # flip PC2

In [ ]:
pca_all = PCA(n_components=20)
pca_all.fit(adata_hvg.layers['zlognorm'][...]) #
pcs_all = pca_all.transform(adata_hvg.layers['zlognorm'][...])

pcs_all[:,0] = -pcs_all[:,0] # flip PC1
pcs_all[:,1] = -pcs_all[:,1] # flip PC1

In [ ]:
# %%time
# ucs_all = UMAP(n_components=2, n_neighbors=30).fit_transform(pcs_all) #

In [ ]:
vt = pca_early.components_
topgenes_early = genes_sel[np.flip(np.argsort(np.abs(vt), axis=1), axis=1)]
print(topgenes_early[:,:8])

vt = pca_later.components_
topgenes_later = genes_sel[np.flip(np.argsort(np.abs(vt), axis=1), axis=1)]
print(topgenes_later[:,:8])

In [ ]:
adata.obsm['pca_early'] = pcs_early
adata.obsm['pca_later'] = pcs_later
adata.obsm['pca_all']   = pcs_all

In [ ]:
res0 = pd.DataFrame(adata.layers['zlognorm'][...], columns=genes)
res0['cond'] = conds
res0['type'] = types
res0['samp'] = samps
res0['rep']  = [samp[-1] for samp in samps]
res0['type'] = np.char.add('c', res0['type'].values.astype(str))

res1 = pd.DataFrame(pcs_early, columns=np.char.add("pcs_early", ((1+np.arange(pcs_early.shape[1])).astype(str))))
res2 = pd.DataFrame(pcs_later, columns=np.char.add("pcs_later", ((1+np.arange(pcs_later.shape[1])).astype(str))))
res3 = pd.DataFrame(pcs_all, columns=np.char.add("pcs_all", ((1+np.arange(pcs_all.shape[1])).astype(str))))
res = pd.concat([res0, res1, res2, res3], axis=1)
# res4 = pd.DataFrame(ucs_all, columns=np.char.add("ucs_all", ((1+np.arange(ucs_all.shape[1])).astype(str))))
# res = pd.concat([res0, res1, res2, res3, res4], axis=1)

In [ ]:
allcolors = sns.color_palette('tab20b', 20)
allcolors

In [ ]:
allcolors2 = sns.color_palette('tab10', 20)
allcolors2

In [ ]:
palette = collections.OrderedDict({
     "P6": allcolors[1],
     "P8": allcolors[0],
    "P10": allcolors[4+2],
    "P12": allcolors[4+1],
    "P14": allcolors[4+0],
    
    "P17": allcolors[8+2],
    "P21": allcolors[8+0],
    
    "P12DR": allcolors[8+2],
    "P14DR": allcolors[8+0],
    "P17DR": allcolors[8+0],
    "P21DR": allcolors[8+0],
    
})
cases = np.array(list(palette.keys()))

cond_order_dict = {
    'P6':  0,
    'P8':  1,
    'P10': 2,
    'P12': 3,
    'P14': 4,
    'P17': 5,
    'P21': 6,
    
    'P12DR': 7,
    'P14DR': 8,
    'P17DR': 9,
    'P21DR': 10,
}
unq_conds = np.array(list(cond_order_dict.keys()))
adata.obs['cond_order'] = adata.obs['cond'].apply(lambda x: cond_order_dict[x])

palette_types = collections.OrderedDict({
    'L2/3_A': allcolors2[0],
    'L2/3_B': allcolors2[1],
    'L2/3_C': allcolors2[2],
})             

palette_types = {
    'c14': 'C0', 
    'c18': 'C1',
    'c16': 'C2', 
    
    'c13': 'C0', 
    'c15': 'C1', 
    'c17': 'C2',
}
type_order = [key for key, val in palette_types.items()]
type_order

In [ ]:
from scipy import stats
from matplotlib.ticker import MaxNLocator

def plot(x, y, aspect_equal=False, density=False, hue='type'):
    n = len(cases)
    fig, axs = plt.subplots(1,n,figsize=(4*n,4*1), sharex=True, sharey=True)
    for i, (ax, cond) in enumerate(zip(axs.flat, cases)):
        ax.set_title(cond)
        sns.scatterplot(data=res, 
                        x=x, y=y, 
                        c='lightgray',
                        s=1, edgecolor='none', 
                        legend=False,
                        ax=ax,
                        rasterized=True,
                       )
        if hue == 'type':
            sns.scatterplot(data=res[res['cond']==cond].sample(frac=1, replace=False),
                            x=x, y=y, 
                            hue='type',
                            hue_order=list(palette_types.keys()),
                            palette=palette_types,
                            s=3, edgecolor='none', 
                            legend=False,
                            ax=ax,
                            rasterized=True,
                           )
        else:
            sns.scatterplot(data=res[res['cond']==cond].sample(frac=1, replace=False),
                            x=x, y=y, 
                            hue='rep',
                            # hue_order=list(palette_types.keys()),
                            # palette=palette_types,
                            s=3, edgecolor='none', 
                            legend=False,
                            ax=ax,
                            rasterized=True,
                           )
            
        if density:
            sns.histplot(data=res[res['cond']==cond],
                            x=x, y=y, 
                            legend=False,
                            ax=ax,
                            rasterized=True,
                           )
        sns.despine(ax=ax)
        ax.xaxis.set_major_locator(MaxNLocator(nbins=3))
        ax.yaxis.set_major_locator(MaxNLocator(nbins=3))
        if aspect_equal:
            ax.set_aspect('equal')
        if i > 0:
            ax.set_xlabel('')
            ax.set_ylabel('')
    return fig
    # plt.show()
    
def plot2(x, y, hue=None, aspect_equal=False, s=10):
    n = len(cases)
    fig, axs = plt.subplots(1,n,figsize=(4*n,4*1), sharex=True, sharey=True)
    fig.suptitle(hue, x=0, ha='left')
    for i, (ax, cond) in enumerate(zip(axs.flat, cases)):
        ax.set_title(cond)
        # sns.scatterplot(data=res, 
        #                 x=x, y=y, 
        #                 c='lightgray',
        #                 alpha=0.3,
        #                 s=1, edgecolor='none', 
        #                 legend=False,
        #                 ax=ax,
        #                 rasterized=True,
        #                )
        show = res[res['cond']==cond]
        if hue:
            # order = np.argsort(show[hue].values)
            ax.scatter(
                       # show[x].iloc[order], show[y].iloc[order], c=show[hue].iloc[order], 
                       show[x], show[y], c=show[hue], 
                       cmap='coolwarm',
                       vmin=-2.5, vmax=2.5,
                       s=s, 
                       edgecolor='none', 
                       rasterized=True,
                      )
        else:
            r, p = stats.spearmanr(show[x], show[y])
            ax.scatter(show[x], show[y],  
                       s=s, 
                       edgecolor='none', 
                       rasterized=True,
                      )
            ax.set_title(f'{cond}\n r={r:.2f}')
        sns.despine(ax=ax)
        ax.xaxis.set_major_locator(MaxNLocator(nbins=3))
        ax.yaxis.set_major_locator(MaxNLocator(nbins=3))
        if aspect_equal:
            ax.set_aspect('equal')
        if i > 0:
            ax.set_xlabel('')
            ax.set_ylabel('')
        ax.grid(False)
    fig.tight_layout()
    
    return fig
    # plt.show()

# Plot A vs C genes aligning cells along early vs late PCs

In [ ]:
palette_time = sns.cubehelix_palette(n_colors=7, start=.5, rot=-.5)
palette_time

In [ ]:
dfplot = res[~res['cond'].str.contains('DR')]
dfplot = dfplot.sample(frac=1, replace=False)

fig, ax = plt.subplots(1,1,figsize=(8,6))
sns.scatterplot(data=dfplot, 
                x='pcs_all1', y='pcs_all2', hue='cond', 
                hue_order=list(palette.keys())[:7],
                palette=palette_time, # sns.cubehelix_palette(start=.5, rot=-.5, 7), # 'rocket', # 'Set2', #palette,
                s=10,
                edgecolor='none',
                rasterized=True,
                ax=ax)
ax.grid(False)
sns.despine(ax=ax)
ax.legend(bbox_to_anchor=(1,1))
output = os.path.join(outdirfig, f'pcall12.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
genes = ['Meis2', 'Foxp1', 'Zbtb20']
for gn in genes:
    fig, ax = plt.subplots(1,1,figsize=(4,4))
    sns.scatterplot(data=dfplot, 
                    x='pcs_all1', y='pcs_all2', #hue='Meis2', 
                    s=10,
                    edgecolor='none',
                    rasterized=True,
                    c=dfplot[gn].values,
                    cmap='coolwarm',
                    vmin=-2.5, 
                    vmax=2.5,
                    ax=ax)
    ax.grid(False)
    sns.despine(ax=ax)
    ax.legend(bbox_to_anchor=(1,1))
    plt.show()

In [ ]:
plot('pcs_all1', 'pcs_all2', aspect_equal=True)

In [ ]:
plot('pcs_early1', 'pcs_early2', aspect_equal=True)
plot('pcs_later1', 'pcs_later2', aspect_equal=True)

In [ ]:
# fig = plot2('pcs_all1', 'pcs_all2', hue='', aspect_equal=True, s=20)

In [ ]:
# plot2('pcs_early1', 'pcs_early2', hue='Pcdh7', aspect_equal=True)
fig = plot2('pcs_later1', 'pcs_later2', hue='Meis2', aspect_equal=True, s=20)

In [ ]:
genes = ['Meis2', 'Foxp1', 'Nfib', 'Rfx3', 'Sox5', 'Zbtb20', 'Tox', 'Tshz3']
for gn in genes:
    fig = plot2('pcs_later1', 'pcs_later2', hue=gn, aspect_equal=True, s=20)
    output = os.path.join(outdirfig, f'pc12_{gn}.pdf')
    powerplots.savefig_autodate(fig, output)
    plt.show()
    

# Archetypes

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from scipy import sparse 
from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
from py_pcha import PCHA

In [ ]:
def norm(x, depths):
    """
    Arguments: 
        x - cell by gene count matrix
        depths - sequencing depth per cell
        
    Output:
        xn - normalized count matrix

    This function takes raw counts as the input, and does the following steps sequencially.
         1. size normalization (CP10k) 
         2. log1p normalization (base 2 - log2(1+CP10k))
         3. zscore per gene  
    """

    xn = x/depths.reshape(-1,1)*1e4
    xn = np.log2(1+xn)
    xn = zscore(xn, axis=0)

    if np.any(np.isnan(xn)):
        print('Warning: the normalized matrix contains nan values. Check input.')
        xn = np.nan_to_num(xn, 0)

    return xn

In [ ]:
def proj(x_norm, ndim, method='PCA'):
    """
    Arguments: 
        x_norm - normalized cell by gene feature matrix
        ndim   - number of dimensions

    Output:
        x_proj - a low-dimensional representation of `x_norm` 

    Here we only implemented PCA - a common projection method widely used, including by
    Adler et al. 2019 and Xie et al. 2024 for the Achetypal Analysis of scRNA-seq data.

    In principle, one can also choose to use other projection methods as needed.
    """

    if method == 'PCA':
        x_proj = PCA(n_components=ndim).fit_transform(x_norm)
    else:
        raise ValueError('methods other than PCA are not implemented...')

    return x_proj


In [ ]:
def pcha(X, noc=3, delta=0, **kwargs):
    """
    """
    XC, S, C, SSE, varexpl = PCHA(X, noc=noc, delta=delta, **kwargs)
    XC = np.array(XC)
    XC = XC[:,np.argsort(XC[0])] # assign an order according to x-axis 
    return XC 

In [ ]:
def downsamp(x, which='cell', p=0.8, seed=None):
    """
    Arguments:
        x - cell by gene matrix
        which - downsample cells (rows) or genes (columns)
        p - fraction of cells/genes to keep - should be a value between ~ [0,1]
    """
    n0, n1 = x.shape
    
    rng = np.random.default_rng(seed=seed)
    
    if which in [0, 'cell', 'row']:
        return x[rng.random(n0)<p, :]
    elif which in [1, 'gene', 'col', 'column']:
        return x[:, rng.random(n1)<p]
    else:
        raise ValueError('choose from cell or gene')

In [ ]:
def shuffle_rows_per_col(x, seed=None):
    """
    Arguments:
       x - cell by gene matrix
       seed - a random seed for reproducibility
    
    shuffles entries across rows (cells) independently for each col (gene)
    """
    rng = np.random.default_rng(seed=seed)
    x_shuff = rng.permuted(x, axis=0)
    return x_shuff

In [ ]:
def plot_archetype(ax, aa, fmt='--o', color='k', **kwargs):
    """
    """
    ax.plot(aa[0].tolist()+[aa[0,0]], aa[1].tolist()+[aa[1,0]], fmt, color=color, **kwargs)

In [ ]:
def get_t_ratio(xp, aa):
    """
    Arguments:
     xp -- projected matrix (cell by 2)
     aa -- inferred archetypes (2 by noc)
     note that this function only works for 2-dimensional space only
     
    Return: 
     t-ratio - ratio of areas (convex hull vs PCH)
     
    """
    assert xp.shape[1] == aa.shape[0] == 2
    
    ch_area  = ConvexHull(xp).volume
   
    x = aa[0]
    y = aa[1]
    pch_area = 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

    return ch_area/pch_area 

In [ ]:
class SingleCellArchetype():
    """
    """
    def __init__(self, x, depths, types):
        """
        Arguments: 
            x - cell by gene count matrix
            depths - sequencing depth per cell
            types  - cell type labels per cell
        
        Initiate the SingleCellArchetype object

        """
        
        # input
        self.x = x
        self.depths = depths
        self.types = types
        
        # cell type label
        types_idx, types_lbl = pd.factorize(types, sort=True)
        
        self.types_idx = types_idx
        self.types_lbl = types_lbl 
        
        # normalize
        self.xn = norm(self.x, self.depths)
        
        # feature matrix 
        self.xf = None 
        return 
        
    def setup_feature_matrix(self, method='data'):
        """
        """
        if method == 'data': 
            self.xf = self.xn
            print('use data')
            return  
        
        elif method == 'gshuff':
            # shuffle gene expression globally across all cells
            self.xf = shuffle_rows_per_col(self.xn)
            print('use shuffled data')
            return
            
        elif method == 'tshuff':
            # shuff each gene across cells independently - internally for each type A,B,C
            xn = self.xn
            xn_tshuff = xn.copy()
            
            types_lbl = self.types_lbl
            types_idx = self.types_idx
            for i in range(len(types_lbl)):
                xn_tshuff[types_idx==i] = shuffle_rows_per_col(xn[types_idx==i])
            self.xf = xn_tshuff
            print('use per-type shuffled data')
            return
        else:
            raise ValueError('choose from (data, gshuff, tshuff)')
    
    def proj_and_pcha(self, ndim, noc, **kwargs):
        """
        """
        xp = proj(self.xf, ndim)
        aa = pcha(xp.T, noc=noc, **kwargs)
        
        self.xp = xp
        self.aa = aa
        return (xp, aa)
        
    def downsamp_proj_pcha(self, ndim, noc, nrepeats=10, which='cell', p=0.8, **kwargs): 
        """
        """
        aa_dsamps = []
        for i in range(nrepeats):
            xn_dsamp = downsamp(self.xf, which=which, p=p)
            xp_dsamp = proj(xn_dsamp, ndim)
            aa_dsamp = pcha(xp_dsamp.T, noc=noc, **kwargs)
            aa_dsamps.append(aa_dsamp)
            
        return aa_dsamps
    
    def t_ratio_test(self, ndim, noc, nrepeats=10, **kwargs): 
        """
        this only work for 2-dimensional space for now
        """
        assert ndim == 2
        
        self.setup_feature_matrix(method='data')
        xp, aa = self.proj_and_pcha(ndim, noc)
        t_ratio = get_t_ratio(xp, aa)
        
        t_ratios_shuff = []
        for i in range(nrepeats):
            self.setup_feature_matrix(method='gshuff')
            xp_shuff, aa_shuff = self.proj_and_pcha(ndim, noc)
            t_ratio_shuff = get_t_ratio(xp_shuff, aa_shuff)
            t_ratios_shuff.append(t_ratio_shuff)
            
        pvalue = (np.sum(t_ratio > t_ratios_shuff)+1)/nrepeats
        
        return t_ratio, t_ratios_shuff, pvalue

In [ ]:
ndim = 2
noc = 3
delta = 0.2 # 0

xp = res[res['cond']=='P21'][['pcs_later1', 'pcs_later2']].values # pcs_later[:,:2]
aa = pcha(xp.T, noc=noc, delta=delta)

# plot
fig, ax = plt.subplots(1,1,figsize=(8,6))
ax.scatter(xp[:,0], xp[:,1], s=2,)
           # c=types_colorvec)
    
plot_archetype(ax, aa, fmt='-o', color='k', zorder=0)
ax.set_title('data')
    
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_aspect('equal')
sns.despine(ax=ax)
ax.grid(False)
    
plt.show()

In [ ]:
todo_conds = [
    'P12DR', 'P14DR', 'P17DR', 'P21DR',
    'P6', 'P8', 'P10', 'P12', 'P14', 'P17', 'P21', 
]

In [ ]:
ndim = 2
noc = 3
delta = 0.2 # 0

aas = []

for cond in todo_conds:
    print(cond)
    xp = res[res['cond']==cond][['pcs_later1', 'pcs_later2']].values # pcs_later[:,:2]
    aa = pcha(xp.T, noc=noc, delta=delta)
    aas.append(aa)
    

In [ ]:
# plot
n = len(todo_conds)
fig, axs = plt.subplots(1,n,figsize=(8*n,6), sharex=True, sharey=True)
for i, cond in enumerate(todo_conds):
    ax = axs[i]
    aa = aas[i]
    xp = res[res['cond']==cond][['pcs_later1', 'pcs_later2']].values # pcs_later[:,:2]
    
    ax.scatter(xp[:,0], xp[:,1], s=2) #  c=types_colorvec, s=2)
    plot_archetype(ax, aa, fmt='-o', color='k', zorder=0)
    ax.set_title(cond)

    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_aspect('equal')
    sns.despine(ax=ax)
    ax.grid(False)
    
plt.show()

In [ ]:
# plot
n = len(todo_conds)
fig, axs = plt.subplots(1,n,figsize=(8*n,6), sharex=True, sharey=True)
for i, cond in enumerate(todo_conds):
    ax = axs[i]
    aa = aas[-1]
    xp = res[res['cond']==cond][['pcs_later1', 'pcs_later2']].values # pcs_later[:,:2]
    
    ax.scatter(xp[:,0], xp[:,1], s=2) #  c=types_colorvec, s=2)
    plot_archetype(ax, aa, fmt='-o', color='k', zorder=0)
    ax.set_title(cond)

    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_aspect('equal')
    sns.despine(ax=ax)
    ax.grid(False)
    
plt.show()

In [ ]:
xbins = np.linspace(-6,6,2*12+1)
ybins = np.linspace(-5,5,2*10+1)

xbins, ybins

In [ ]:
from scipy.stats import gaussian_kde



In [ ]:
# fit KDE

kdes = []
kde_values = []
n = len(todo_conds)
for i, cond in enumerate(todo_conds):
    xp = res[res['cond']==cond][['pcs_later1', 'pcs_later2']].values # pcs_later[:,:2]
    
    # Fit KDE
    kde = gaussian_kde(xp.T)
    kdes.append(kde)
    kde_values.append(kde(xp.T))
    


In [ ]:
# plot
n = len(todo_conds)
fig, axs = plt.subplots(1,n,figsize=(4*n,3), sharex=True, sharey=True)
for i, cond in enumerate(todo_conds):
    ax = axs[i]
    # kde = kdes[i]
    kde_val = kde_values[i]
    aa = aas[-1]
    
    xp = res[res['cond']==cond][['pcs_later1', 'pcs_later2']].values # pcs_later[:,:2]
    
    # calc KDE
    # c = kde(xp.T)
    c = kde_val
    
    # histogram
#     hist, _, _ = np.histogram2d(xp[:,0], xp[:,1], bins=[xbins, ybins])
#     hist = hist.T[::-1]
    
#     # sns.heatmap(hist, ax=ax)
#     ax.imshow(hist/np.sum(hist), 
#               extent=(-6,6,-5,5), 
#               cmap='gray_r',
#               )

    ax.scatter(xp[:,0], xp[:,1], c=c, s=10, cmap='rocket_r', rasterized=True) #  c=types_colorvec, s=2)
    
    plot_archetype(ax, aa, fmt='--', color='gray', zorder=2)
    ax.scatter(aa[0,0], aa[1,0], color='C0', zorder=2)
    ax.scatter(aa[0,1], aa[1,1], color='C1', zorder=2)
    ax.scatter(aa[0,2], aa[1,2], color='C2', zorder=2)
    
    ax.set_title(cond)

    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_aspect('equal')
    sns.despine(ax=ax)
    ax.grid(False)
    # break
    
output = os.path.join(outdirfig, f'pc12_density.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:

genes = ['Meis2', 'Foxp1', 'Nfib', 'Rfx3', 'Sox5', 'Zbtb20', 'Tox', 'Tshz3']
for gn in genes:
    fig = plot2('pcs_later1', 'pcs_later2', hue=gn, aspect_equal=True, s=20)
    axs = fig.get_axes()
    for ax in axs:
        # plot_archetype(ax, aa, fmt='--o', color='gray', zorder=2)
        plot_archetype(ax, aa, fmt='--', color='gray', zorder=2)
        # ax.scatter(aa[0,0], aa[1,0], color='C0', zorder=2)
        # ax.scatter(aa[0,1], aa[1,1], color='C1', zorder=2)
        # ax.scatter(aa[0,2], aa[1,2], color='C2', zorder=2)
    
    output = os.path.join(outdirfig, f'pc12_{gn}.pdf')
    powerplots.savefig_autodate(fig, output)
    plt.show()
    
    # break

# JS distance heatmap

In [ ]:
from scipy.spatial.distance import jensenshannon
from scipy.cluster import hierarchy as sch

In [ ]:
bins = np.arange(-6, 6+1/2, 1/2)
print(bins)
    
histograms = []
for i, cond in enumerate(todo_conds):
    xp = res[res['cond']==cond][['pcs_later1', 'pcs_later2']].values # pcs_later[:,:2]
    a, _, _ = np.histogram2d(xp[:,0], xp[:,1], bins=bins)
    a = a/np.sum(a)*100
    histograms.append(a)

In [ ]:
num = len(todo_conds)
res_js = np.zeros((num,num))
js_min = 1
js_max = 0
for i in np.arange(num):
    for j in np.arange(num):
        if j > i:
            p = np.ravel(histograms[i])
            q = np.ravel(histograms[j])
            p = p/np.sum(p)
            q = q/np.sum(q)
            js = jensenshannon(p, q) 
            
            res_js[i,j] = js
            res_js[j,i] = js
            js_min = min(js, js_min)
            js_max = max(js, js_max)

In [ ]:
order, Z = basicu.order_by_hc(res_js, return_Z=True) # [::-1]

In [ ]:
js_min, js_max

In [ ]:
fig, axs = plt.subplots() # subplot_mosaic("ABBBB", figsize=(10,5))
ax = axs

# ax = axs['B']
sns.heatmap(pd.DataFrame(res_js[:4, 4:]), cmap='Blues_r', 
            vmin=js_min,
            vmax=0.5,
            cbar_kws=dict(shrink=0.5, ticks=[0.2, 0.5], label='Jensen-Shannon distance'),
            yticklabels=np.array(todo_conds)[:4],
            xticklabels=np.array(todo_conds)[4:],
            ax=ax,
           )
ax.set_aspect('equal')
output = os.path.join(outdirfig, "js_heatmap_nrdr_only_1.pdf")
powerplots.savefig_autodate(fig, output)

In [ ]:
fig, ax = plt.subplots()
ax.plot(res_js[0,4:])
ax.plot(res_js[1,4:])
ax.plot(res_js[2,4:])
ax.plot(res_js[3,4:])
ax.set_xticks(np.arange(7))
ax.set_xticklabels(['6', '8', '10', '12', '14', '17', '21'])

In [ ]:
fig, axs = plt.subplots() # subplot_mosaic("ABBBB", figsize=(10,5))
ax = axs

# ax = axs['B']
sns.heatmap(pd.DataFrame(res_js[4:, 4:]), cmap='Blues_r', 
            vmin=0,
            vmax=js_max,
            cbar_kws=dict(shrink=0.5, ticks=[0.2, 0.5], label='Jensen-Shannon distance'),
            yticklabels=np.array(todo_conds)[4:],
            xticklabels=np.array(todo_conds)[4:],
            ax=ax,
           )
ax.set_aspect('equal')
output = os.path.join(outdirfig, "js_heatmap_nrdr_only.pdf")
powerplots.savefig_autodate(fig, output)

In [ ]:
fig, axs = plt.subplot_mosaic("ABBBB", figsize=(10,5))

ax = axs['A']
d = sch.dendrogram(Z, ax=ax, orientation='left', above_threshold_color='k', color_threshold=0)
sns.despine(ax=ax, left=True, bottom=True)
ax.invert_yaxis()
ax.grid(False)

ax = axs['B']
sns.heatmap(pd.DataFrame(res_js).reindex(order)[order], cmap='Blues_r', 
            # vmin=js_min, vmax=0.4,
            vmin=js_min,
            # vmin=0.2,
            # vmax=0.3,
            cbar_kws=dict(shrink=0.5, label='Jensen-Shannon distance'),
            xticklabels=np.array(todo_conds)[order],
            yticklabels=np.array(todo_conds)[order],
            ax=ax,
           )
ax.set_aspect('equal')
output = os.path.join(outdirfig, "js_heatmap_nrdr_only_0.pdf")
powerplots.savefig_autodate(fig, output)

In [ ]:
fig, axs = plt.subplot_mosaic("BBBB", figsize=(10,5))


ax = axs['B']
sns.heatmap(pd.DataFrame(res_js), cmap='Blues_r', 
            vmin=js_min,
            cbar_kws=dict(shrink=0.5, ticks=[0.2, 0.3], label='Jensen-Shannon distance'),
            xticklabels=np.array(todo_conds),
            yticklabels=np.array(todo_conds),
            ax=ax,
           )
ax.set_aspect('equal')
# output = os.path.join(outdirfig, "js_heatmap_nrdr_only.pdf")
# powerplots.savefig_autodate(fig, output)

In [ ]:
neworder = [4,5,6,0,7,1,8,2,9,3,10]

fig, axs = plt.subplot_mosaic("BBBB", figsize=(10,5))


ax = axs['B']
sns.heatmap(pd.DataFrame(res_js).reindex(neworder)[neworder], cmap='Blues_r', 
            vmin=js_min,
            cbar_kws=dict(shrink=0.5, ticks=[0.3, 0.7], label='Jensen-Shannon distance'),
            xticklabels=np.array(todo_conds)[neworder],
            yticklabels=np.array(todo_conds)[neworder],
            ax=ax,
           )
ax.set_aspect('equal')
# output = os.path.join(outdirfig, "js_heatmap_nrdr_only.pdf")
# powerplots.savefig_autodate(fig, output)

# variance explained by PCs

In [ ]:
# variance along PC1 and PC2
var0s = []
var1s = []
for i, cond in enumerate(todo_conds):
    xp = res[res['cond']==cond][['pcs_later1', 'pcs_later2']].values # pcs_later[:,:2]
    var0 = np.std(xp[:,0])
    var1 = np.std(xp[:,1])

    var0s.append(var0)
    var1s.append(var1)

In [ ]:
var0_max = np.max(var0s)
var1_max = np.max(var1s)

fig, axs = plt.subplots(1,2,figsize=(2*5,1*4))
ax = axs[0]
ax.plot(np.arange(7)     , var0s[4:]/var0_max)
ax.plot(np.arange(7)[-4:], var0s[:4]/var0_max)
ax.set_ylim([0,1])

ax = axs[1]
ax.plot(np.arange(7)     , var1s[4:]/var1_max, '-o')
ax.plot(np.arange(7)[-4:], var1s[:4]/var1_max, '-o')
ax.set_ylim([0,1])
plt.show()

# variance along A-C axis vs B-axis

In [ ]:
# define A-C (t0) and B (t1) axis

t0 = aa[:,2]-aa[:,0]
t0 = t0/np.sqrt(t0.dot(t0))
t1 = np.array([t0[1], -t0[0]])

print(np.sum(t0**2), t0.dot(t1))

In [ ]:
fjg, ax = plt.subplots()
plot_archetype(ax, aa, fmt='--', color='gray', zorder=2)
ax.plot([aa[0,0],aa[0,0]+t0[0]], [aa[1,0], aa[1,0]+t0[1]])
ax.plot([aa[0,0],aa[0,0]+t1[0]], [aa[1,0], aa[1,0]+t1[1]])
ax.set_aspect('equal')

In [ ]:
# variance along PC1 and PC2
vart0s = []
vart1s = []
for i, cond in enumerate(todo_conds):
    xp = res[res['cond']==cond][['pcs_later1', 'pcs_later2']].values # pcs_later[:,:2]
    xpt0 = xp.dot(t0)
    xpt1 = xp.dot(t1)
    
    vart0 = np.std(xpt0)
    vart1 = np.std(xpt1)

    vart0s.append(vart0)
    vart1s.append(vart1)

In [ ]:
times = np.array([6,8,10,12,14,17,21])

In [ ]:
vart0_max = np.max(vart0s)
vart1_max = np.max(vart1s)

fig, axs = plt.subplots(1,2,figsize=(2*5,1*4), sharex=True, sharey=True)
ax = axs[0]
ax.plot(times     , vart0s[4:]/vart0_max, color='C1')
ax.plot(times[-4:], vart0s[:4]/vart0_max, color='k')
sns.despine(ax=ax)

ax = axs[1]
ax.plot(times     , vart1s[4:]/vart1_max, color='C1')
ax.plot(times[-4:], vart1s[:4]/vart1_max, color='k')
sns.despine(ax=ax)
ax.set_ylim([0,1.05])
ax.set_xticks(times)
plt.show()

# add replicates